In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import json
import requests
import time
import datetime
import warnings
warnings.filterwarnings('ignore')



# создаём функцию получения данных по курсу доллар - биткоин
def get_data(max_date, amount=2000):
    from_cur = 'BTC'
    to_cur = 'USD'
    data = pd.DataFrame()
    interval='hour'
        
    limit = '&limit=' + str(amount)
    max_date = datetime.datetime.strptime(max_date, "%Y-%m-%d")
    max_date = int(time.mktime(max_date.timetuple()))
    upto = '&toTs=' + str(max_date)
    
    url = 'https://min-api.cryptocompare.com/data/histo' + interval + '?fsym=' + from_cur + '&tsym=' + to_cur + limit + upto
    try:
        res = requests.get(url)
        data = pd.DataFrame(res.json()['Data'])
    except:
        print(res)
        raise ConnectionError('Connection error')  
    data['time'] = data['time'].apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime('%Y-%m-%dT%H:%M:%SZ'))
    return data

# фунция вычисления ema
def get_ema(df, ema, target, column):
    df[column] = np.nan
    # первое данное за период определяем как простое скользящее
    df[column][ema-1] = df[df.index <= ema-1][target].mean()
    # вычисляем alpha
    alpha = 2 / (ema + 1)
    # вычисляем ema
    for i in range(ema,len(df)):
        df[column][i] = alpha * df[target][i] + (1 - alpha) * df[column][i-1]
    df[column] = df[column].apply(lambda x: round(x,2))
    return df

# функция вычисления sma
def get_sma(df, sma, target, column):
    df[column] = np.nan
    # первое данное за период определяем как простое скользящее
    for i in range(sma+df[~pd.isnull(df[target])].index[0],len(df)):
        df[column][i] = df[(df.index <= i-1) & (df.index > i-1-sma)][target].mean()
    df[column] = df[column].apply(lambda x: round(x,2))
    return df


# устанавливаем параметры бота

def bot(ema_ps = 12, ema_pl = 26, signal_pa = 9):
    # получаем шапку датафрейма
    df = get_data(str(datetime.datetime.now().date()), amount=1)
    df.drop(axis=0, labels=[0,1], inplace=True)

    # определяем границы временного интервала и указатель позиции
    start_date = datetime.datetime(2018, 1, 1)
    end_date = datetime.datetime.now()
    position = end_date

    # указываем интервал времени для выкачки (есть лимит выкачки за один запрос)
    days_int = 30
    delta = datetime.timedelta(days=days_int)

    # выкачиваем данные
    while (position - delta) > start_date:
    #    print(position)
        df_temp = get_data(str(position.date()), amount=days_int*24)
        df = pd.concat([df,df_temp])
        position -= delta
    else:
        days_int = int(round((position - start_date).total_seconds() / 60 / 60 / 24,0))
        df_temp = get_data(str(position.date()), amount=days_int*24)
        df = pd.concat([df,df_temp])

    # удаляем дубликаты, если есть, преобразуем формат времени, дропаем лишние данные, сортируем
    df.drop_duplicates(subset=['time'], keep='first')
    df['time'] = df['time'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%dT%H:%M:%SZ'))
    df = df[df['time']>=start_date]
    df = df.sort_values(by='time', ascending=True).reset_index(drop=True)

    # создаём столбцы c быстрой и медленной экспоненциальной скользящей
    df = get_ema(df, ema=ema_ps, target='close', column='ema_ps')
    df = get_ema(df, ema=ema_pl, target='close', column='ema_pl')

    # создаём MACD
    df['MACD'] = df['ema_pl'] - df['ema_ps']
    # создаём сигнальную линию
    df = get_sma(df, sma=signal_pa, target='MACD', column='signal')

    # создаём столбец с индикатором сигнальной линии выше/ниже MACD
    df['intercept'] = df['signal'] - df['MACD']
    df['intercept'] = df['intercept'].apply(lambda x: 1 if (x > 0) else(-1 if (x < 0) else (np.nan if np.isnan(x) else 0)))  

    # создаём столбец с индикацией пересечения MACD и сигнальной линии
    df['trigger'] = np.nan
    for i in range(df[~pd.isnull(df['signal'])].index[0]+1,len(df)):
        df['trigger'][i] = df['intercept'][i] - df['intercept'][i-1]

    # вычисляем результат на основе двух условий
    df['result'] = '-'
    df['result'] = df.apply(lambda x: 'вход' if ((x['MACD'] > 0) and (x['trigger'] > 0)) else
                            ('выход' if ((x['MACD'] < 0) and (x['trigger'] < 0)) else x['result']), axis=1)

    # делаем сдвиг - осуществление операции на следующем шаге после получения сигнала
    df['result'].index = df['result'].index+1
    df['result'][0]='-'
    df = pd.concat([df['time'], df['close'], df['open'], df['result']], axis=1)

    # оставляем строки с данными по открытию/закрытию
    df = df[df['result'] != '-']

    # получаем путь к испоняемому файлу
    path = os.path.abspath(os.getcwd())

    # сохраняем csv
    df.to_csv(os.path.join(path,'test3.csv'), sep=';', index=False)
#    print('результат сохранён '+os.path.join(path,'test3.csv'))
    
bot()